# User's Guide, Chapter 13: More Music21Object Attributes and Properties

At this point you know how to find a `Music21Object`, how to name them and group them (with `.id` and `.groups`) and how to position them in Streams (with `.offset`, `.priority`, `.classSortOrder` and the `.activeSite`). This section gets first into some more advanced things that Music21Objects have, then some more fun things.

### Sites and the storing of elements

All `Music21Objects` (i.e., elements) have a `.sites` property which is a :class:`~music21.sites.Sites` object which holds information about all the places the `Music21Object` is stored in. At its simplest, it's something that can be iterated over:

In [1]:
n = note.Note()
s1 = stream.Stream(id='s1')
s2 = stream.Stream(id='s2')
s1.insert(10, n)
s2.insert(20, n)

for s in n.sites:
    print(s, s.elementOffset(n))

<music21.stream.Stream s1> 10.0
<music21.stream.Stream s2> 20.0


Note that the order of the Streams in `.sites` is guaranteed to be the order in which the note was inserted into the site.

### Derivations

We will talk about derivations more in a future chapter, butwe alluded to them in the Example in chapter 10, so let's say a few words about this advanced feature. A :class:`~music21.derivation.Derivation` object is a pointer to an object that this object is derived from in some way.  They've gone their separate ways to an extent, but may want to talk to each other later.  A `Music21Object` starts out with no useful Derivation:

In [2]:
c = note.Note("C4")
c.derivation

<Derivation of <music21.note.Note C> from None via "None">

But we can create new notes from it and they're not totally connected, but show their connection through `.derivation`:

In [3]:
f = c.transpose('P4')
f

<music21.note.Note F>

In [4]:
f.derivation

<Derivation of <music21.note.Note F> from <music21.note.Note C> via "transpose">

Now `c` has a life of its own from `f`.  We can add a sharp to C and the transpose relationship of F to C does not affect it:

In [5]:
c.pitch.accidental = pitch.Accidental('sharp')
(c, f)

(<music21.note.Note C#>, <music21.note.Note F>)

But if `f` wants to do something to `c`, it can by changing itself and every element of its `.derivation.chain()`:

In [6]:
f.notehead = 'diamond'
for n in f.derivation.chain():
    n.notehead = 'diamond'

(f.notehead, c.notehead)

('diamond', 'diamond')